In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Retrieve QLD traffic data that has average traffic data by hour and by day in 2023.
qld_traffic_df = pd.read_excel('Resources/qld-traffic-data-average-by-hour-by-day-2023.xlsx')
qld_traffic_df.head()

,SITE_ID,DESCRIPTION,SITE_DISTANCE,LONGITUDE,LATITUDE,RSECT_ID,ROAD_NAME,TDIST_START,TDIST_END,GAZETTAL_DIRECTION,HOURS,MON,TUE,WED,THU,FRI,SAT,SUN,WEEKDAY_AVERAGE,WEEKEND_AVERAGE
0,131646,""" Sth of Myora Res- Dunwich""",2.5,153.411081,-27.482689,1000,"""EAST COAST ROAD""",0.0,9.36,AGAINST GAZETTAL,0 to 1,0.0,1.0,0.0,0.0,1.0,2,4.0,0,3
1,131646,""" Sth of Myora Res- Dunwich""",2.5,153.411081,-27.482689,1000,"""EAST COAST ROAD""",0.0,9.36,AGAINST GAZETTAL,1 to 2,1.0,0.0,0.0,0.0,1.0,2,1.0,0,2
2,131646,""" Sth of Myora Res- Dunwich""",2.5,153.411081,-27.482689,1000,"""EAST COAST ROAD""",0.0,9.36,AGAINST GAZETTAL,2 to 3,0.0,0.0,0.0,2.0,0.0,1,1.0,0,1
3,131646,""" Sth of Myora Res- Dunwich""",2.5,153.411081,-27.482689,1000,"""EAST COAST ROAD""",0.0,9.36,AGAINST GAZETTAL,3 to 4,1.0,1.0,1.0,1.0,1.0,1,1.0,1,1
4,131646,""" Sth of Myora Res- Dunwich""",2.5,153.411081,-27.482689,1000,"""EAST COAST ROAD""",0.0,9.36,AGAINST GAZETTAL,4 to 5,6.0,6.0,6.0,8.0,6.0,2,1.0,6,1


In [3]:
qld_traffic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84240 entries, 0 to 84239
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   SITE_ID             84240 non-null  int64  
 1   DESCRIPTION         84240 non-null  object 
 2   SITE_DISTANCE       84240 non-null  float64
 3   LONGITUDE           84240 non-null  float64
 4   LATITUDE            84240 non-null  float64
 5   RSECT_ID            84240 non-null  object 
 6   ROAD_NAME           84240 non-null  object 
 7   TDIST_START         84240 non-null  float64
 8   TDIST_END           84240 non-null  float64
 9   GAZETTAL_DIRECTION  84240 non-null  object 
 10  HOURS               84240 non-null  object 
 11  MON                 84192 non-null  float64
 12  TUE                 84192 non-null  float64
 13  WED                 84096 non-null  float64
 14  THU                 83952 non-null  float64
 15  FRI                 84192 non-null  float64
 16  SAT 

In [4]:
# Find site_id value for M1 highway
# M1 Highway is also called Pacific Highway
m1_traffic_df = qld_traffic_df[qld_traffic_df['ROAD_NAME'].str.contains(r'(M1|Pacific).*(Highway|Motorway)', case=False, na=False)]
m1_traffic_df

/var/folders/4c/qxymt3y16qj5kt7d4m_0kkk00000gn/T/ipykernel_1857/2208056647.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m1_traffic_df = qld_traffic_df[qld_traffic_df['ROAD_NAME'].str.contains(r'(M1|Pacific).*(Highway|Motorway)', case=False, na=False)]


,SITE_ID,DESCRIPTION,SITE_DISTANCE,LONGITUDE,LATITUDE,RSECT_ID,ROAD_NAME,TDIST_START,TDIST_END,GAZETTAL_DIRECTION,HOURS,MON,TUE,WED,THU,FRI,SAT,SUN,WEEKDAY_AVERAGE,WEEKEND_AVERAGE
12144,131796,"""Pac Mwy sth side Paradise Rd overpass""",5.90,153.140291,-27.639688,12A,"""PACIFIC HIGHWAY (PACIFIC MOTORWAY)""",2.60,6.54,AGAINST GAZETTAL,0 to 1,259.0,234.0,251.0,250.0,268.0,359,477.0,252,417
12145,131796,"""Pac Mwy sth side Paradise Rd overpass""",5.90,153.140291,-27.639688,12A,"""PACIFIC HIGHWAY (PACIFIC MOTORWAY)""",2.60,6.54,AGAINST GAZETTAL,1 to 2,211.0,198.0,217.0,220.0,236.0,265,334.0,216,299
12146,131796,"""Pac Mwy sth side Paradise Rd overpass""",5.90,153.140291,-27.639688,12A,"""PACIFIC HIGHWAY (PACIFIC MOTORWAY)""",2.60,6.54,AGAINST GAZETTAL,2 to 3,246.0,248.0,256.0,261.0,264.0,230,251.0,255,240
12147,131796,"""Pac Mwy sth side Paradise Rd overpass""",5.90,153.140291,-27.639688,12A,"""PACIFIC HIGHWAY (PACIFIC MOTORWAY)""",2.60,6.54,AGAINST GAZETTAL,3 to 4,560.0,570.0,587.0,571.0,583.0,327,269.0,574,298
12148,131796,"""Pac Mwy sth side Paradise Rd overpass""",5.90,153.140291,-27.639688,12A,"""PACIFIC HIGHWAY (PACIFIC MOTORWAY)""",2.60,6.54,AGAINST GAZETTAL,4 to 5,1895.0,1959.0,1983.0,1901.0,1802.0,716,444.0,1908,581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81115,136454,"""U12A- North of Gaza Road Overpass PTC""",8.55,153.058027,-27.533219,U12A,"""SOUTH-EAST ARTERIAL ROAD (PACIFIC MOTORWAY)""",7.65,10.42,GAZETTAL,19 to 20,2142.0,2499.0,2649.0,2807.0,2791.0,2442,2047.0,2581,2239
81116,136454,"""U12A- North of Gaza Road Overpass PTC""",8.55,153.058027,-27.533219,U12A,"""SOUTH-EAST ARTERIAL ROAD (PACIFIC MOTORWAY)""",7.65,10.42,GAZETTAL,20 to 21,1823.0,2126.0,2246.0,2398.0,2458.0,2425,1894.0,2215,2152
81117,136454,"""U12A- North of Gaza Road Overpass PTC""",8.55,153.058027,-27.533219,U12A,"""SOUTH-EAST ARTERIAL ROAD (PACIFIC MOTORWAY)""",7.65,10.42,GAZETTAL,21 to 22,1696.0,1997.0,2176.0,2307.0,2717.0,2645,1829.0,2190,2225
81118,136454,"""U12A- North of Gaza Road Overpass PTC""",8.55,153.058027,-27.533219,U12A,"""SOUTH-EAST ARTERIAL ROAD (PACIFIC MOTORWAY)""",7.65,10.42,GAZETTAL,22 to 23,1184.0,1473.0,1632.0,1879.0,2496.0,2690,1308.0,1750,1980


In [5]:
print(m1_traffic_df['ROAD_NAME'].unique())
print(m1_traffic_df['RSECT_ID'].unique())

['"PACIFIC HIGHWAY (PACIFIC MOTORWAY)"'
 '"PACIFIC HIGHWAY CONNECTION ROAD"'
 '"SOUTH-EAST ARTERIAL ROAD (PACIFIC MOTORWAY)"']
['12A' 906 'U12A']


The Pacific Motorway starts from Brisbane CBD. For this analysis, I've set the range from Springwood (Exit No.19) to Burleigh Heads (Exit No.89).

- **12A** refers to the M1 Motorway segment connecting Logan and the NSW/QLD border.
- **U12A** is the section from the CBD to Logan (before 12A).
- **906** connects the Logan Sub-Arterial Road and 12A.

Therefore, the correct data range for this analysis is **12A**.

In [6]:
m1_traffic_df = m1_traffic_df[(m1_traffic_df['RSECT_ID']=='12A') & (m1_traffic_df['GAZETTAL_DIRECTION']=='GAZETTAL')].reset_index(drop=True)
m1_traffic_df

,SITE_ID,DESCRIPTION,SITE_DISTANCE,LONGITUDE,LATITUDE,RSECT_ID,ROAD_NAME,TDIST_START,TDIST_END,GAZETTAL_DIRECTION,HOURS,MON,TUE,WED,THU,FRI,SAT,SUN,WEEKDAY_AVERAGE,WEEKEND_AVERAGE
0,131796,"""Pac Mwy sth side Paradise Rd overpass""",5.9,153.140291,-27.639688,12A,"""PACIFIC HIGHWAY (PACIFIC MOTORWAY)""",2.6,6.54,GAZETTAL,0 to 1,403.0,440.0,446.0,474.0,539.0,722,760.0,460,741
1,131796,"""Pac Mwy sth side Paradise Rd overpass""",5.9,153.140291,-27.639688,12A,"""PACIFIC HIGHWAY (PACIFIC MOTORWAY)""",2.6,6.54,GAZETTAL,1 to 2,248.0,303.0,310.0,319.0,375.0,453,466.0,311,460
2,131796,"""Pac Mwy sth side Paradise Rd overpass""",5.9,153.140291,-27.639688,12A,"""PACIFIC HIGHWAY (PACIFIC MOTORWAY)""",2.6,6.54,GAZETTAL,2 to 3,244.0,298.0,314.0,324.0,348.0,338,338.0,306,338
3,131796,"""Pac Mwy sth side Paradise Rd overpass""",5.9,153.140291,-27.639688,12A,"""PACIFIC HIGHWAY (PACIFIC MOTORWAY)""",2.6,6.54,GAZETTAL,3 to 4,374.0,402.0,430.0,446.0,460.0,357,321.0,422,339
4,131796,"""Pac Mwy sth side Paradise Rd overpass""",5.9,153.140291,-27.639688,12A,"""PACIFIC HIGHWAY (PACIFIC MOTORWAY)""",2.6,6.54,GAZETTAL,4 to 5,995.0,1037.0,1078.0,1090.0,1070.0,627,478.0,1054,553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,12058,"""WiM Site Tugun Bypass""",79.0,153.494730,-28.159915,12A,"""PACIFIC HIGHWAY (PACIFIC MOTORWAY)""",76.3,79.23,GAZETTAL,19 to 20,705.0,730.0,830.0,975.0,1088.0,749,690.0,866,719
212,12058,"""WiM Site Tugun Bypass""",79.0,153.494730,-28.159915,12A,"""PACIFIC HIGHWAY (PACIFIC MOTORWAY)""",76.3,79.23,GAZETTAL,20 to 21,499.0,529.0,587.0,708.0,833.0,637,508.0,632,572
213,12058,"""WiM Site Tugun Bypass""",79.0,153.494730,-28.159915,12A,"""PACIFIC HIGHWAY (PACIFIC MOTORWAY)""",76.3,79.23,GAZETTAL,21 to 22,351.0,393.0,444.0,566.0,643.0,544,369.0,480,456
214,12058,"""WiM Site Tugun Bypass""",79.0,153.494730,-28.159915,12A,"""PACIFIC HIGHWAY (PACIFIC MOTORWAY)""",76.3,79.23,GAZETTAL,22 to 23,234.0,261.0,304.0,373.0,461.0,469,242.0,327,354


**Gazettal direction** refers to the direction of traffic flow.  
The **Pacific Highway** starts from the **CBD** side and heads toward the **Gold Coast**. Since I mostly drive on this side of the road, I focused on analyzing it. Therefore, I extracted data with the relevant **Gazettal** value.

In [9]:
sites = list(m1_traffic_df['SITE_ID'].unique())
sites

[131796, 136399, 135793, 13006, 12205, 11641, 11399, 160229, 12058]

In [11]:
# Changing 'Hours' column values to starting time
# '0 to 1' -> '0'
# '23 to 24' -> '23'
m1_traffic_df['HOURS'] = m1_traffic_df['HOURS'].apply(lambda x: x.split()[0])
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   SITE_ID             216 non-null    int64  
 1   DESCRIPTION         216 non-null    object 
 2   SITE_DISTANCE       216 non-null    float64
 3   LONGITUDE           216 non-null    float64
 4   LATITUDE            216 non-null    float64
 5   RSECT_ID            216 non-null    object 
 6   ROAD_NAME           216 non-null    object 
 7   TDIST_START         216 non-null    float64
 8   TDIST_END           216 non-null    float64
 9   GAZETTAL_DIRECTION  216 non-null    object 
 10  HOURS               216 non-null    object 
 11  MON                 216 non-null    float64
 12  TUE                 216 non-null    float64
 13  WED                 216 non-null    float64
 14  THU                 216 non-null    float64
 15  FRI                 216 non-null    float64
 16  SAT     